In [3]:
!pip install requests feedparser matplotlib wordcloud nltk gradio praw scikit-learn spacy
!python -m nltk.downloader stopwords
!python -m nltk.downloader vader_lexicon
!python -m spacy download en_core_web_sm


<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 97.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecti

In [4]:
import requests
import feedparser
import io
import base64
from datetime import datetime, timedelta
from collections import Counter, defaultdict
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import gradio as gr
import praw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy
import numpy as np
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

GNEWS_API_KEY = "46a3e5dbc2ac726efc117a977791d605"
NEWSAPI_KEY = "6e6471621d62400d8a81a61799b715e9"
reddit = praw.Reddit(
    client_id="RK5_wfgydhs-NaIJ1y6pqg",
    client_secret="aZTZKB4Aq2af1DbCIJNY3g3sh2X4Yg",
    user_agent="SatyaSrikarScript"
)

def get_start_time(period):
    now = datetime.utcnow()
    if period == "Day":
        return now - timedelta(days=1)
    elif period == "Week":
        return now - timedelta(weeks=1)
    elif period == "Month":
        return now - timedelta(days=30)
    elif period == "Year":
        return now - timedelta(days=365)
    return now

def analyze_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'Positive', score, 'green'
    elif score <= -0.05:
        return 'Negative', score, 'red'
    return 'Neutral', score, 'gray'

def img_html(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches='tight')
    buf.seek(0)
    encoded = base64.b64encode(buf.read()).decode()
    buf.close()
    plt.close()
    return f'<img src="data:image/png;base64,{encoded}"/>'

def generate_wordcloud(posts):
    text = " ".join(p["title"] for p in posts)
    wc = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    return img_html(plt)

def plot_sentiment_distribution(posts):
    counts = Counter(p['sentiment_category'] for p in posts)
    labels, values = zip(*counts.items()) if counts else ([], [])
    plt.figure(figsize=(6, 6))
    plt.pie(values, labels=labels, autopct='%1.1f%%', colors=['green', 'gray', 'red'])
    return img_html(plt)

def plot_top_words(posts):
    words = " ".join(p['title'] for p in posts).lower().split()
    filtered = [w for w in words if w not in stop_words and len(w) > 1]
    most_common = Counter(filtered).most_common(10)
    if not most_common:
        return "<b>No keywords to display.</b>"
    labels, values = zip(*most_common)
    plt.figure(figsize=(8, 5))
    plt.bar(labels, values, color='skyblue')
    return img_html(plt)

def topic_modeling(posts):
    texts = [p['title'] for p in posts]
    if len(texts) < 3:
        return "<b>Insufficient data for topic modeling.</b>"
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=2, random_state=42)
    lda.fit(X)
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic in lda.components_:
        topic_words = [feature_names[i] for i in topic.argsort()[:-6:-1]]
        topics.append(", ".join(topic_words))
    return "<b>Topics:</b><br>" + "<br>".join([f"- {t}" for t in topics])

def extract_keywords(text):
    doc = nlp(text)
    keywords = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return ", ".join(keywords)

def extract_named_entities(text):
    doc = nlp(text)
    entities = defaultdict(int)
    for ent in doc.ents:
        entities[ent.text] += 1
    return "<br>".join([f"{ent}: {count}" for ent, count in entities.items()])

def calculate_influence_score(sentiment_score, engagement):
    return sentiment_score + np.log(1 + engagement) * 0.1

def fetch_reddit(keyword, period):
    start_time = get_start_time(period)
    posts = []
    subreddits = ['all', 'worldnews', 'technology', 'sports', 'news', 'science', 'business', 'politics']
    for subreddit in subreddits:
        for submission in reddit.subreddit(subreddit).search(keyword, sort='new', limit=50):
            post_time = datetime.utcfromtimestamp(submission.created_utc)
            if post_time < start_time:
                continue
            text = submission.title + " " + (submission.selftext or "")
            sent_cat, sent_score, sent_color = analyze_sentiment(text)
            engagement = submission.score
            influence_score = calculate_influence_score(sent_score, engagement)
            posts.append({
                'title': submission.title,
                'source': 'Reddit',
                'published': str(post_time),
                'url': submission.url,
                'sentiment_category': sent_cat,
                'sentiment_score': sent_score,
                'sentiment_color': sent_color,
                'influence_score': influence_score
            })
    return posts

def fetch_gnews(keyword, period):
    start_time = get_start_time(period)
    url = f"https://gnews.io/api/v4/search?q={keyword}&lang=en&sortby=publishedAt&max=50&apikey={GNEWS_API_KEY}"
    res = requests.get(url).json()
    posts = []
    for a in res.get("articles", []):
        try:
            pub_date = datetime.strptime(a['publishedAt'], "%Y-%m-%dT%H:%M:%SZ")
            if pub_date < start_time:
                continue
        except:
            continue
        text = a['title'] + " " + a.get("description", "")
        sent_cat, sent_score, sent_color = analyze_sentiment(text)
        engagement = a.get("popularity", 0)
        influence_score = calculate_influence_score(sent_score, engagement)
        posts.append({
            'title': a['title'],
            'source': a['source']['name'],
            'published': a['publishedAt'],
            'url': a['url'],
            'sentiment_category': sent_cat,
            'sentiment_score': sent_score,
            'sentiment_color': sent_color,
            'influence_score': influence_score
        })
    return posts

def fetch_newsapi(keyword, period):
    start_time = get_start_time(period)
    url = f"https://newsapi.org/v2/everything?q={keyword}&language=en&sortBy=publishedAt&pageSize=100&apiKey={NEWSAPI_KEY}"
    res = requests.get(url).json()
    posts = []
    for a in res.get("articles", []):
        try:
            pub_date = datetime.strptime(a['publishedAt'], "%Y-%m-%dT%H:%M:%SZ")
            if pub_date < start_time:
                continue
        except:
            continue
        text = a['title'] + " " + a.get("description", "")
        sent_cat, sent_score, sent_color = analyze_sentiment(text)
        engagement = a.get("popularity", 0)
        influence_score = calculate_influence_score(sent_score, engagement)
        posts.append({
            'title': a['title'],
            'source': a['source']['name'],
            'published': a['publishedAt'],
            'url': a['url'],
            'sentiment_category': sent_cat,
            'sentiment_score': sent_score,
            'sentiment_color': sent_color,
            'influence_score': influence_score
        })
    return posts

def fetch_rss(keyword, period):
    start_time = get_start_time(period)
    urls = [
        "https://rss.cnn.com/rss/cnn_topstories.rss",
        "https://feeds.bbci.co.uk/news/rss.xml",
        "https://techcrunch.com/feed/"
    ]
    posts = []
    for url in urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            try:
                pub_date = datetime(*entry.published_parsed[:6])
                if pub_date < start_time:
                    continue
            except:
                continue
            if keyword.lower() in entry.title.lower():
                sent_cat, sent_score, sent_color = analyze_sentiment(entry.title)
                influence_score = calculate_influence_score(sent_score, len(entry.title))
                posts.append({
                    'title': entry.title,
                    'source': 'RSS Feed',
                    'published': entry.published,
                    'url': entry.link,
                    'sentiment_category': sent_cat,
                    'sentiment_score': sent_score,
                    'sentiment_color': sent_color,
                    'influence_score': influence_score
                })
    return posts

def fetch_all(keyword, period):
    posts = []
    posts += fetch_reddit(keyword, period)
    posts += fetch_gnews(keyword, period)
    posts += fetch_newsapi(keyword, period)
    posts += fetch_rss(keyword, period)
    return posts

def generate_classification_report(posts):
    true_labels = []
    predicted_labels = []

    for p in posts:
        true_labels.append(p['sentiment_category'])
        predicted_labels.append(p['sentiment_category'])

    report = classification_report(true_labels, predicted_labels, target_names=['Positive', 'Neutral', 'Negative'])

    return report

def format_output(posts):
    if not posts:
        return "<b>No results found.</b>"

    html = "<h3>🗳️ Top Influential Posts</h3>"
    posts = sorted(posts, key=lambda x: x['influence_score'], reverse=True)[:50]

    for i, p in enumerate(posts, 1):
        html += f"""
        <div style='padding:10px;border-bottom:1px solid #ccc;'>
            <b>#{i}</b> <a href='{p['url']}' target='_blank'>{p['title']}</a><br>
            📅 {p['published']} | 📰 {p['source']}<br>
            ❤️ Sentiment: <span style='color:{p['sentiment_color']};'>{p['sentiment_category']}</span>
        </div>
        """

    html += "<br><h4>☁️ Word Cloud</h4>" + generate_wordcloud(posts)
    html += "<h4>📊 Sentiment Distribution</h4>" + plot_sentiment_distribution(posts)
    html += "<h4>🔠 Frequent Words</h4>" + plot_top_words(posts)
    html += "<h4>🧠 Topic Modeling</h4>" + topic_modeling(posts)
    html += "<h4>🔑 Extracted Keywords</h4>" + ", ".join([extract_keywords(p['title']) for p in posts])
    html += "<h4>🧬 Named Entities</h4>" + "<br>".join([extract_named_entities(p['title']) for p in posts])

    # Generate the classification report
   # classification_rep = generate_classification_report(posts)
   # html += f"<h4>📊 Sentiment Classification Report</h4><pre>{classification_rep}</pre>"

    return html

def main_func(keyword, period):
    posts = fetch_all(keyword, period)
    return format_output(posts)

# Gradio UI
interface = gr.Interface(
    fn=main_func,
    inputs=[
        gr.Textbox(label="Keyword or Topic"),
        gr.Dropdown(choices=["Day", "Week", "Month", "Year"], label="Time Period")
    ],
    outputs=gr.HTML(label="Trending Insights"),
    title="🧠 Real-Time Detection of Trending Topics via Multi-Source Aggregation and Natural Language Processing",

    description="Aggregates content from Reddit, GNews, NewsAPI, and RSS. Provides sentiment analysis, influence scoring, word cloud, keywords, NER, and topic modeling."
)

interface.launch(share=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8035072d21aacf5192.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
